## openai 初始化

In [ ]:
%pip install openai
%pip install python-dotenv
%pip install pydantic

In [1]:
import os
from dotenv import load_dotenv
from typing import Union, Optional, List
load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
# print(f"OpenAI API Key: {openai_api_key}")

In [2]:
from openai import OpenAI
client = OpenAI()


# 廢話留言tag

### 第一層級的tag
instruction: "你是YouTube留言內容分析研究員，分類留言是否為「無意義留言」。"

In [9]:
from openai import OpenAI
from pydantic import BaseModel

client = OpenAI()

def useless_comment_test1(text):
    instructions = """
    你是YouTube留言內容分析研究員，分類留言是否為「無意義留言」。
    只能用繁體中文回答

    <output>
    output as a json object:
    {
        "tag": bool,  # true=無意義、false=有意義
        "reason": str  # 無意義留言的原因
    }
    """

    class CommentAnalysis(BaseModel):
        tag: bool  # 是否為無意義留言
        reason: str  # 無意義留言的原因

    try:
        response = client.responses.parse(
            model="gpt-4.1-nano-2025-04-14",
            input=[
                {"role": "system", "content": instructions},
                {"role": "user", "content": text}
            ],
            text_format=CommentAnalysis,
        )

        result = response.output_parsed

        result_dict = {
            "tag": result.tag,
            "reason": result.reason
        }
        print(result_dict)
        return result_dict
    
    except Exception as e:
        print(f"Error: {e}")
        return {"tag": False, "reason": f"分析失敗: {str(e)}"}

In [10]:
## main function
def test1(comment_list):
    test1_result = []
    for comment in comment_list:
        print(f"Comment: {comment}")
        result = useless_comment_test1(comment)
        print(f"Result: {result}\n")

        temp_dict = {
            "comment_text" : comment,
            "tag": result['tag'],
            "reason": result['reason']
        }
        test1_result.append(temp_dict)

    return test1_result

In [ ]:
comment_list = [
    '大陆官员都是韩国俞',
    '谁会想到 王牌变成了新闻乱报',
    '第二季來了 我來回顧我最喜歡的新聞亂報 希望下一季可以安全玩政治',
    '求 女生 IG FB',
    '等等 那張車馬費我想要',
    '攝影機大哥是不是只故意拍妹子XD',
    '知聰明近乎勇',
    '模仿原汁那一段真是原汁原味',
    '結尾call back太好笑了',
    '打老二 跟大富翁好好笑',
    '檢查無效 司法有理',
    '結尾好屌',
    '結果天上掉下一席立委給元之',
    '真好笑 幫App廣告 不忘酸一下政治人物 那請問 你們酸人家 有分他廣告費嗎 那天 有人酸 後籠 賣產品 你應該也不會跟他拿 遮羞費吧 哈哈'
]

result = test1(comment_list)

In [ ]:
# save result as csv
import pandas as pd

test1_df = pd.DataFrame(result)
# filename = f"gpt_tag/useless_tag_v{i}.csv"
filename = "gpt_tag/useless_test1.csv"
test1_df.to_csv(filename, index=False, encoding='utf-8-sig')
print(f"Results saved to {filename}")

Results saved to gpt_tag/useless_test1.csv


### 第二層級的tag
instruction加上pseudo labeling


In [21]:
def useless_comment_test2(text):
    instructions = """
    你是YouTube留言內容分析研究員，分類留言是否為「無意義留言」。
    只能用繁體中文回答

    <definition>
    無意義留言是指那些沒有實質內容、無法提供有價值的資訊或討論的留言。
    </definition>

    <examples>
    - "求 女生 IG FB",  tag=true
    - "可憐的奮蛆們", tag=true
    - "真的 不好笑", tag=true
    - "賴的老家是國民黨遷台之前就有的 你們這節目一直批評賴的老家有沒有良心", tag=flase
    - "網路投票就不知道是不是個人意願啦 也沒辦法做到不公開吧", tag=false 
    - "所以說前幾年說要發放的數位身分證在哪裡呢", tag=false
    </examples>

    <output>
    output as a json object:
    {
        "tag": bool,  # true=無意義、false=有意義
        "reason": str  # 無意義留言的原因
    }
    """

    class CommentAnalysis(BaseModel):
        tag: bool  # 是否為無意義留言
        reason: str  # 無意義留言的原因

    try:
        response = client.responses.parse(
            model="gpt-4.1-nano-2025-04-14",
            input=[
                {"role": "system", "content": instructions},
                {"role": "user", "content": text}
            ],
            text_format=CommentAnalysis,
        )

        result = response.output_parsed

        result_dict = {
            "tag": result.tag,
            "reason": result.reason
        }
        print(result_dict)
        return result_dict
    
    except Exception as e:
        print(f"Error: {e}")
        return {"tag": False, "reason": f"分析失敗: {str(e)}"}

In [23]:
## main function
def test2(comment_list):
    test2_result = []
    for comment in comment_list:
        print(f"Comment: {comment}")
        result = useless_comment_test2(comment)
        print(f"Result: {result}\n")

        temp_dict = {
            "comment_text" : comment,
            "tag": result['tag'],
            "reason": result['reason']
        }
        test2_result.append(temp_dict)

    return test2_result

In [24]:
# test comments
comment_list = [
    '大陆官员都是韩国俞',
    '谁会想到 王牌变成了新闻乱报',
    '第二季來了 我來回顧我最喜歡的新聞亂報 希望下一季可以安全玩政治',
    '求 女生 IG FB',
    '等等 那張車馬費我想要',
    '攝影機大哥是不是只故意拍妹子XD',
    '知聰明近乎勇',
    '模仿原汁那一段真是原汁原味',
    '結尾call back太好笑了',
    '打老二 跟大富翁好好笑',
    '檢查無效 司法有理',
    '結尾好屌',
    '結果天上掉下一席立委給元之',
    '真好笑 幫App廣告 不忘酸一下政治人物 那請問 你們酸人家 有分他廣告費嗎 那天 有人酸 後籠 賣產品 你應該也不會跟他拿 遮羞費吧 哈哈'
]

result2 = test2(comment_list)

Comment: 大陆官员都是韩国俞
{'tag': True, 'reason': '留言內容缺乏具體資訊或討論點，屬於無意義留言。'}
Result: {'tag': True, 'reason': '留言內容缺乏具體資訊或討論點，屬於無意義留言。'}

Comment: 谁会想到 王牌变成了新闻乱报
{'tag': False, 'reason': '留言具有討論性，涉及對某事件或情況的看法，具有一定的意義'}
Result: {'tag': False, 'reason': '留言具有討論性，涉及對某事件或情況的看法，具有一定的意義'}

Comment: 第二季來了 我來回顧我最喜歡的新聞亂報 希望下一季可以安全玩政治
{'tag': False, 'reason': '留言表達了對內容的期待與希望，具有討論意義，不是無意義留言。'}
Result: {'tag': False, 'reason': '留言表達了對內容的期待與希望，具有討論意義，不是無意義留言。'}

Comment: 求 女生 IG FB
{'tag': True, 'reason': '留言內容沒有實質討論或有用資訊，只是詢問聯絡方式，屬於無意義留言。'}
Result: {'tag': True, 'reason': '留言內容沒有實質討論或有用資訊，只是詢問聯絡方式，屬於無意義留言。'}

Comment: 等等 那張車馬費我想要
{'tag': False, 'reason': '留言表達了對車馬費的需求，具有一定的內容與意圖，屬於有意義的留言。'}
Result: {'tag': False, 'reason': '留言表達了對車馬費的需求，具有一定的內容與意圖，屬於有意義的留言。'}

Comment: 攝影機大哥是不是只故意拍妹子XD
{'tag': False, 'reason': '留言具有討論內容，關於攝影和拍攝對象的話題，具有一定的意義。'}
Result: {'tag': False, 'reason': '留言具有討論內容，關於攝影和拍攝對象的話題，具有一定的意義。'}

Comment: 知聰明近乎勇
{'tag': False, 'reason': '該留言具有一定的哲理意義，表達了智慧與勇氣的關聯性，屬於有意義的留言。'}
Result: {'tag': 

In [27]:
# add tag result to useless_test1.csv, add column "tag2" and "reason2"
test2_df = pd.DataFrame(result2)
test2_df.rename(columns={"tag": "tag2", "reason": "reason2"}, inplace=True)
# test2_df.head(5)

# combine test1_df and test2_df
combined_df = pd.merge(test1_df, test2_df, on="comment_text", how="outer")
combined_df.head(5)

,comment_text,tag,reason,tag2,reason2
0,大陆官员都是韩国俞,True,留言內容不具備明確的意義，似乎為不相關或無意義的陳述。,True,留言內容缺乏具體資訊或討論點，屬於無意義留言。
1,谁会想到 王牌变成了新闻乱报,False,留言內容具有一定程度的意義，表達對某事件或情況的看法或感受。,False,留言具有討論性，涉及對某事件或情況的看法，具有一定的意義
2,第二季來了 我來回顧我最喜歡的新聞亂報 希望下一季可以安全玩政治,False,留言內容表達了對節目的喜好與期待，具有一定的意義和評論性。,False,留言表達了對內容的期待與希望，具有討論意義，不是無意義留言。
3,求 女生 IG FB,True,留言內容簡單，沒有提供具體資訊或有建設性的內容，屬於無意義留言。,True,留言內容沒有實質討論或有用資訊，只是詢問聯絡方式，屬於無意義留言。
4,等等 那張車馬費我想要,False,留言內容涉及對車馬費的需求，具有特定意義和實質內容。,False,留言表達了對車馬費的需求，具有一定的內容與意圖，屬於有意義的留言。


### 第三層級的tag
instruction加上指標，reasonu要說明符合哪些指標

In [32]:
def useless_comment_test3(text):
    instructions = """
    你是YouTube留言內容分析研究員，分類留言是否為「無意義留言」。
    只能用繁體中文回答

    <definition>
    無意義留言的指標：
    1. 留言少於5個字
    2. 留言內容沒有意義 e.g. 哈哈哈笑死、真的 不好笑
    3. 留言內容重複 e.g. 主持人出來道歉 歧視弱勢團體 罷看 主持人出來道歉 歧視弱勢團體 罷看...
    4. 情緒性強或引發反感 e.g. 可憐的奮蛆們、賀龍神精病三八女、廢物綠蛆 廢物藍叫
    </definition>

    <examples>
    - "求 女生 IG FB",  tag=true
    - "可憐的奮蛆們", tag=true
    - "主持人出來道歉 歧視弱勢團體 罷看 主持人出來道歉 歧視弱勢團體 罷看...", tag=true
    - "賴的老家是國民黨遷台之前就有的 你們這節目一直批評賴的老家有沒有良心", tag=flase
    - "網路投票就不知道是不是個人意願啦 也沒辦法做到不公開吧", tag=false 
    - "所以說前幾年說要發放的數位身分證在哪裡呢", tag=false
    </examples>

    <task>根據definition和examples，分析留言是否為無意義留言</task>
    <dos>
    1. tag: 判斷留言是否符合無意義留言的定義
    2. reason: 說明留言符合哪些definition的指標
    </dos>

    <output>
    output as a json object:
    {
        "tag": bool,  # true=無意義、false=有意義
        "reason": str  # 無意義留言的原因
    }
    """

    class CommentAnalysis(BaseModel):
        tag: bool  # 是否為無意義留言
        reason: str  # 無意義留言的原因

    try:
        response = client.responses.parse(
            model="gpt-4.1-nano-2025-04-14",
            input=[
                {"role": "system", "content": instructions},
                {"role": "user", "content": text}
            ],
            text_format=CommentAnalysis,
        )

        result = response.output_parsed

        result_dict = {
            "tag": result.tag,
            "reason": result.reason
        }
        print(result_dict)
        return result_dict
    
    except Exception as e:
        print(f"Error: {e}")
        return {"tag": False, "reason": f"分析失敗: {str(e)}"}

In [33]:
## main function
def test3(comment_list):
    test3_result = []
    for comment in comment_list:
        print(f"Comment: {comment}")
        result = useless_comment_test3(comment)
        print(f"Result: {result}\n")

        temp_dict = {
            "comment_text" : comment,
            "tag": result['tag'],
            "reason": result['reason']
        }
        test3_result.append(temp_dict)

    return test3_result

In [35]:
# test comments
comment_list = [
    '大陆官员都是韩国俞',
    '谁会想到 王牌变成了新闻乱报',
    '第二季來了 我來回顧我最喜歡的新聞亂報 希望下一季可以安全玩政治',
    '求 女生 IG FB',
    '等等 那張車馬費我想要',
    '攝影機大哥是不是只故意拍妹子XD',
    '知聰明近乎勇',
    '模仿原汁那一段真是原汁原味',
    '結尾call back太好笑了',
    '打老二 跟大富翁好好笑',
    '檢查無效 司法有理',
    '結尾好屌',
    '結果天上掉下一席立委給元之',
    '真好笑 幫App廣告 不忘酸一下政治人物 那請問 你們酸人家 有分他廣告費嗎 那天 有人酸 後籠 賣產品 你應該也不會跟他拿 遮羞費吧 哈哈'
]

result3 = test3(comment_list)

Comment: 大陆官员都是韩国俞
{'tag': False, 'reason': '留言內容有意義，提到具體人物且表達觀點，不符合無意義留言的指標'}
Result: {'tag': False, 'reason': '留言內容有意義，提到具體人物且表達觀點，不符合無意義留言的指標'}

Comment: 谁会想到 王牌变成了新闻乱报
{'tag': False, 'reason': '留言內容有意義，表達對新聞報導的不滿或感慨，符合有意義留言的特徵'}
Result: {'tag': False, 'reason': '留言內容有意義，表達對新聞報導的不滿或感慨，符合有意義留言的特徵'}

Comment: 第二季來了 我來回顧我最喜歡的新聞亂報 希望下一季可以安全玩政治
{'tag': False, 'reason': '留言內容較完整，表達對節目的期待與評論，符合有意義留言的範圍'}
Result: {'tag': False, 'reason': '留言內容較完整，表達對節目的期待與評論，符合有意義留言的範圍'}

Comment: 求 女生 IG FB
{'tag': True, 'reason': '留言內容少於5個字，沒有明確的意義，屬於無意義留言'}
Result: {'tag': True, 'reason': '留言內容少於5個字，沒有明確的意義，屬於無意義留言'}

Comment: 等等 那張車馬費我想要
{'tag': True, 'reason': '留言內容較短，但沒有明顯的重複或情緒性，符合無意義留言的範例，且內容較為模糊不具意義。'}
Result: {'tag': True, 'reason': '留言內容較短，但沒有明顯的重複或情緒性，符合無意義留言的範例，且內容較為模糊不具意義。'}

Comment: 攝影機大哥是不是只故意拍妹子XD
{'tag': False, 'reason': '留言長度超過5個字，內容涉及對攝影師行為的質疑，具有一定意義，不符合無意義留言的定義。'}
Result: {'tag': False, 'reason': '留言長度超過5個字，內容涉及對攝影師行為的質疑，具有一定意義，不符合無意義留言的定義。'}

Comment: 知聰明近乎勇
{'tag': True, 'reason': '留

In [36]:
# add tag result to useless_test1.csv, add column "tag2" and "reason2"
test3_df = pd.DataFrame(result3)
test3_df.rename(columns={"tag": "tag3", "reason": "reason3"}, inplace=True)
# test2_df.head(5)

# combine test1_df and test2_df
combined_df = pd.merge(combined_df, test3_df, on="comment_text", how="outer")
combined_df.head(5)

,comment_text,tag,reason,tag2,reason2,tag3,reason3
0,大陆官员都是韩国俞,True,留言內容不具備明確的意義，似乎為不相關或無意義的陳述。,True,留言內容缺乏具體資訊或討論點，屬於無意義留言。,False,留言內容有意義，提到具體人物且表達觀點，不符合無意義留言的指標
1,谁会想到 王牌变成了新闻乱报,False,留言內容具有一定程度的意義，表達對某事件或情況的看法或感受。,False,留言具有討論性，涉及對某事件或情況的看法，具有一定的意義,False,留言內容有意義，表達對新聞報導的不滿或感慨，符合有意義留言的特徵
2,第二季來了 我來回顧我最喜歡的新聞亂報 希望下一季可以安全玩政治,False,留言內容表達了對節目的喜好與期待，具有一定的意義和評論性。,False,留言表達了對內容的期待與希望，具有討論意義，不是無意義留言。,False,留言內容較完整，表達對節目的期待與評論，符合有意義留言的範圍
3,求 女生 IG FB,True,留言內容簡單，沒有提供具體資訊或有建設性的內容，屬於無意義留言。,True,留言內容沒有實質討論或有用資訊，只是詢問聯絡方式，屬於無意義留言。,True,留言內容少於5個字，沒有明確的意義，屬於無意義留言
4,等等 那張車馬費我想要,False,留言內容涉及對車馬費的需求，具有特定意義和實質內容。,False,留言表達了對車馬費的需求，具有一定的內容與意圖，屬於有意義的留言。,True,留言內容較短，但沒有明顯的重複或情緒性，符合無意義留言的範例，且內容較為模糊不具意義。


In [37]:
combined_df.to_csv(filename, index=False, encoding='utf-8-sig')
print(f"Results saved to {filename}")

Results saved to gpt_tag/useless_test1.csv


In [ ]:
import pandas as pd
import asyncio
from gpt_instructions import *
from openai import AsyncOpenAI
from pydantic import BaseModel

# 非同步 client
client = AsyncOpenAI()

file_path = "gpt_tag/video_0_useless_tag.csv"

# 定義 Pydantic 類別
class CommentAnalysis(BaseModel):
    tag: bool
    reason: str

# 非同步處理單一 prompt
async def get_useless_response(text, instructions_num):
    try:
        response = await client.chat.completions.create(
            model="gpt-4.1-nano-2025-04-14",
            messages=[
                {"role": "system", "content": instructions_num},
                {"role": "user", "content": text}
            ],
            response_format= {
                "type": "json_object",
            }
        )
        result = CommentAnalysis.model_validate_json(response.choices[0].message.content)
        return {"tag": result.tag, "reason": result.reason}

    except Exception as e:
        return {"tag": False, "reason": f"分析失敗: {str(e)}"}

# 主處理函式：並行執行三組指令
async def process_comments():
    comment_df = pd.read_csv(file_path, encoding='utf-8-sig')[:10]
    async_test_df = comment_df.copy()

    for idx, row in comment_df.iterrows():
        comment_text = row['comment_text']
        print(f"==> 處理第{idx+1}則留言：{comment_text}")

        # 並行處理三組 instructions
        result1, result2, result3 = await asyncio.gather(
            get_useless_response(comment_text, instruction1),
            get_useless_response(comment_text, instruction2),
            get_useless_response(comment_text, instruction3),
        )

        # 更新DataFrame
        async_test_df.at[idx, 'tag1'] = result1['tag']
        async_test_df.at[idx, 'reason1'] = result1['reason']
        async_test_df.at[idx, 'tag2'] = result2['tag']
        async_test_df.at[idx, 'reason2'] = result2['reason']
        async_test_df.at[idx, 'tag3'] = result3['tag']
        async_test_df.at[idx, 'reason3'] = result3['reason']

    # 儲存結果
    async_test_df.to_csv("gpt_tag/video_0_async_tag.csv", index=False, encoding='utf-8-sig')
    print(async_test_df.head(5))

In [21]:
# 同步處理
await process_comments()

處理留言：太有活了贵司
處理留言：我的天这段影片的评论区是发生了什么
處理留言：原来这就是之后节目里提到的被炎上的事件之一吗
處理留言：突然發現低卡的夥伴們ᶘ ᵒᴥᵒᶅ
處理留言：怎麼選舉可以有這麼多好笑的內容
處理留言：桌上是什麼怎麼變成馬賽克了
處理留言：王志安事件
處理留言：原本放什麼
處理留言：為什麼桌上的廠商有打碼 節目結束的廠商沒有打碼
處理留言：藍綠白都臭 只有綠不爽 可見其風度
             comment_text   tag1                          reason1   tag2  \
0                  太有活了贵司   True     內容模糊，不具具體意義或資訊，無法提供有效的反饋或互動。   True   
1       我的天这段影片的评论区是发生了什么  False  留言內容具有疑問或關心影片內容，有意圖交流，不屬於無意義留言。   True   
2  原来这就是之后节目里提到的被炎上的事件之一吗  False         內容具有搜尋或討論的價值，沒有顯示為無意義留言。  False   
3        突然發現低卡的夥伴們ᶘ ᵒᴥᵒᶅ  False      留言內容有表情符號和情感，傳達特定意圖，具有一定的意義   True   
4         怎麼選舉可以有這麼多好笑的內容  False   留言表達對選舉內容的看法，有一定意義，並非純粹的無意義內容。   True   

                               reason2   tag3  \
0           留言內容沒有具體資訊或討論點，屬於模糊或無意義的評論   True   
1  留言內容沒有具體內容或有意義的討論，僅表達困惑或感受，屬於無意義留言。  False   
2   留言內容具有疑問性，涉及對未來節目內容的理解與關注，具備一定討論價值  False   
3            內容缺乏具體資訊或討論價值，只是簡單的表情和口號。   True   
4         留言內容較為空泛，未提供具體資訊或討論，屬於無意義留言。  False   

                                          reason3

In [6]:
# 計算標註一致性

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# 範例資料
true_labels = human_tag_df['human_tag']
pred_labels = human_tag_df['spam_tag']

# 計算四個指標（macro平均適用於不平衡資料）
precision = precision_score(true_labels, pred_labels, average='macro')
recall = recall_score(true_labels, pred_labels, average='macro')
f1 = f1_score(true_labels, pred_labels, average='macro')
accuracy = accuracy_score(true_labels, pred_labels)

print('Precision（精確率）:', precision)
print('Recall（召回率）:', recall)
print('F1 Score:', f1)
print('Accuracy（準確率）:', accuracy)

# 儲存結果
results = {
    "precision": precision,
    "recall": recall,
    "f1_score": f1,
    "accuracy": accuracy
}
results_df = pd.DataFrame([results])
results_df.to_csv("batch_outputs/llm_tagging_metrics.csv", index=False, encoding='utf-8-sig')

Precision（精確率）: 0.845
Recall（召回率）: 0.8672228585053072
F1 Score: 0.8426189825875673
Accuracy（準確率）: 0.845


# 影片相關性

In [26]:
# test 10則

with open('transcripts/v3.txt', 'r', encoding='utf-8') as f:
    transcript = f.read()

print(transcript)

歡迎收看賀瓏夜夜秀
再次感謝冷靜寶貝樂團
在報新聞之前 我想先 shout out 兩個人
卡提諾狂新聞
還有眼球中央電視台
你們辛苦了
但政客們你們可以鬆懈嗎
不行
因為夜夜秀回來了
給我皮繃緊一點政治人物
我們講曾博恩嘛
把一堆覺青叫到現場花錢買票之後
結果是置入
台上的韓國瑜花錢買了置入
那一堆笨蛋 250
坐在底下哈哈笑
今年不一樣了
各位是笨蛋 1000
笨蛋 1200 跟笨蛋 1500
各位 哈哈笑
四年來
很多東西都改變了
包含剛剛提到的朱學恆也改變了
而有些事情
則是我們以為都已經二十年不變了
那二十四年應該也不會變
像是四年前我們在夜夜秀是這樣形容的
今年楚瑜
give up
但也有好的變化
四年前我們在
欸苗栗國的時候當中提到
苗栗連續 26 年不刪預算
隔年 2020 年苗栗終於第一次刪除預算了
而總統候選人們也是改變不大
賴清德
剪影一樣像龜頭
柯文哲
也一樣仇女
郭台銘
一樣被國民黨拋棄
唯一有成長的就是侯友宜
的文大宿舍租金
提到總統候選人
我們來看看最近的民調
賴清德目前民調領先
沒有楚瑜還是很不習慣
在野黨提出的藍白合已經討論超過一個月了
但一定要藍白合嗎
柯文哲民調第二欸
不能單獨參選嗎
從寶傑的反應來看
柯文哲自己當選比
外星人統治地球還難
回到藍白合
我們先來看藍方選手的態度
我不到一秒鐘就決定了
為國家為人民
不管是 Albee 還是吉尼丁
想跟我在一起
我都接受
然後我是第一次看到有人
自己講這是兩情相悅的
看來這樣想的不只有我
你們知道為什麼今天沒有藍白的候選人嗎
他們都在同志大遊行
藍白合看起來是陷入僵局了
準備進入政黨協商
而上週曾經一度傳出郭柯配的消息
但目前看起來唯一願意跟郭台銘配的
只有哩賀阿嬤
至少這個阿嬤比賴佩霞有喜感
除了郭柯配不順之外
郭台銘還有其他的煩惱
被檢舉連署給 400 元之後
接著又爆出了
400 元變 300 元怎麼降價啦
欸各位要幫郭董連署的話趕快去喔
不然照這個趨勢
之後郭台銘連署可能只剩 200 塊
而其中一位涉嫌幫郭買連署書的
竟然是高雄民進黨黨部委員曾睿涵
哇這個選戰才剛開始
已經有內奸了欸
太快跳出來了吧
如果找他去演無間道的話
應該短到只能拍抖音
提到民進黨
我們來看看賴清德對藍白合
和侯友宜的看法
其他候選人都忙著整合
只有賴清德忙著賣慘
在最近活動上
賴清德也提出他對選舉的看

In [28]:
import pandas as pd
comments_df = pd.read_csv("batch_outputs/usefull_comments.csv", encoding='utf-8-sig')
# test_df = comments_df[:50]

# filter video_id == v3
test_df = comments_df[comments_df['video_id'] == 'v3']
test_df = test_df[:50]


In [5]:
video_comment_counts = comments_df.groupby('video_title').size().reset_index(name='comment_count')
print(video_comment_counts)

                        video_title  comment_count
0   【#賀瓏夜夜秀】1/20 新聞亂報 EP9｜落選夜夜秀謝票大會            902
1       【#賀瓏夜夜秀】1/27 新聞亂報 EP10｜新聞自報           7320
2       【#賀瓏夜夜秀】1/6 新聞亂報 EP8｜大選前不演了            752
3     【#賀瓏夜夜秀】10/28 新聞亂報 EP1｜藍白兩情相悅            630
4    【#賀瓏夜夜秀】11/18 新聞亂報 EP3｜沒想到有今天吧            570
5   【#賀瓏夜夜秀】11/25 新聞亂報 EP4｜你也可以不同意啊            854
6     【#賀瓏夜夜秀】11/4 新聞亂報 EP2｜藍白誰攻誰受？            563
7    【#賀瓏夜夜秀】12/16 新聞亂報 EP6｜Blue不錄了           1126
8      【#賀瓏夜夜秀】12/30 新聞亂報 EP7｜新北割侯戰           1102
9        【#賀瓏夜夜秀】12/9 新聞亂報 EP5｜財產亂報            702
10     【#賀瓏夜夜秀】吳欣岱、陳奕齊 最能靠美色競選的本土政黨           3347
11       【#賀瓏夜夜秀】吳欣盈 參選副總統其實有點Shock           5801
12         【#賀瓏夜夜秀】徐巧芯 認定國民黨的男人都很軟爛           1189
13                    【#賀瓏夜夜秀】欸！吹哨者           2827
14                    【#賀瓏夜夜秀】欸！外役監           1194
15                     【#賀瓏夜夜秀】欸！大麻            928
16                    【#賀瓏夜夜秀】欸！投開票           2991
17                    【#賀瓏夜夜秀】欸！特偵組           3142
18                   【#賀瓏夜夜秀】欸！

In [35]:
# 先判斷影片主題關鍵字，再判斷留言相關性？
from pydantic import BaseModel

def fetch_video_keywords(transcript):
    instruction = "只能用繁體中文回答。你是YouTube內容分析研究員，擅長主題分析。萃取10-15組<transcript>的具體關鍵字。關鍵字排除如，賀瓏夜夜秀、冷靜寶貝樂團，以及抽象形容，如：政治喜劇與社會關懷、香港社會"
    class TranscriptKeywords(BaseModel):
        transcript_keywords: list[str]  # 影片主題關鍵字
    try:
        response = client.responses.parse(
            model="gpt-4.1-mini-2025-04-14",
            input=[
                {"role": "system", "content": instruction},
                {"role": "user", "content": transcript}
            ],
            text_format=TranscriptKeywords
        )

        result = response.output_parsed
        print(f"Transcript Keywords: {result.transcript_keywords}")
        return result.transcript_keywords
    
    except Exception as e:
        print(f"Error: {e}")
        return []


In [36]:
transcript_keywords = fetch_video_keywords(transcript)

Transcript Keywords: ['曾博恩', '韓國瑜買票', '苗栗預算刪除', '賴清德民調', '藍白合協商', '郭柯配消息', '郭台銘連署問題', '高雄民進黨內奸', '賴清德女性支持度下降', '台北大巨蛋壓力測試', 'DeFit步數生成APP', '以巴衝突', '李克強心臟病', '淡水阿嬤與蘆洲阿嬤', '賀瓏與哩賀阿嬤']


In [10]:
# load knowledge_list
with open('knowledge_for_relation.txt', 'r', encoding='utf-8') as f:
    knowledge_list = f.read().splitlines()

In [40]:
def comment_video_related(comment, transcript_keywords, knowledge_list):
    instructions = f"""
    必須用繁體中文回答
    你是台灣YouTube留言的內容分析研究員，擅長主題分析，且熟悉台灣的網路政治討論。任務：分析留言是否與影片主題相關。
    
    {knowledge_list}是台灣網路上常見的政治用語 格式 “人物或單位（職稱、節目中角色）：別名”

    # task:
    {transcript_keywords}是影片逐字稿萃取的主題關鍵字。
    請判斷判斷<comment>是否為與影片主題關鍵字討論的內容有關。
    遇到人物、不確定的字詞可以參考knowledge_list後再做判斷。
    空泛的語句、難以從上下文判斷指涉對象，例如「果然面對現實了」，related == false
    具體說明<reason>，例如：留言的“某某”與影片關鍵字“某某”相關。只要列舉1-2個最相關的transcript_keywords。
    
    if related == true, 回傳配對到的transcript_keywords到match_keywords
    else: related為false，match_keywords為None

    <output>
    {{
        "related": bool,
        "reason": str,
        "match_keywords": list[str] | None
    }}
    """

    class CommentAnalysis(BaseModel):
        related: bool  # 是否與影片相關
        reason: str  # 相關的理由
        match_keywords: Optional[List[str]]  # 影片主題關鍵字
    
    try:
        response = client.responses.parse(
            model="gpt-4.1-mini",
            input=[
                {"role": "system", "content": instructions},
                {"role": "user", "content": comment}
            ],
            text_format=CommentAnalysis,
        )

        result = response.output_parsed

        result_dict = {
            "related": result.related,
            "reason": result.reason,
            "match_keywords": result.match_keywords
        }
        print(result_dict)
        return result_dict
    
    except Exception as e:
        print(f"Error: {e}")
        return {"related": False, "reason": f"分析失敗: {str(e)}", "match_keywords": None}

In [41]:
for idx in test_df.index:
    comment = test_df.loc[idx, 'comment']
    print(f"==> 處理第{idx+1}則留言：{comment}")
    result = comment_video_related(comment, transcript,knowledge_list)
    
    test_df.loc[idx, 'related'] = result['related']
    test_df.loc[idx, 'reason'] = result['reason']
    test_df.loc[idx, 'match_keywords'] = ', '.join(result['match_keywords']) if result['match_keywords'] else None
    

==> 處理第8975則留言：現在好多假帳號 還好 這個不是
{'related': False, 'reason': '留言內容提到假帳號，但影片主題是台灣政治人物和選舉民調，留言與影片主題關鍵字不符。', 'match_keywords': None}
==> 處理第8976則留言：大麻能把憂鬱症治好嗎
{'related': False, 'reason': '留言討論大麻治療憂鬱症，與影片中的台灣政治人物、政黨及選舉話題無直接關聯。影片主題關鍵字主要關注政治人物如賴清德、柯文哲、郭台銘等，以及選舉和政治事件，與留言內容無關。', 'match_keywords': None}
==> 處理第8977則留言：果然面對現實了
{'related': False, 'reason': '留言內容過於空泛，無法判斷與影片中任何具體主題關鍵字相關聯。', 'match_keywords': None}
==> 處理第8978則留言：等等等 只有我沒看懂工商嗎 所以他的具體功能是幫你帶跑200公里
{'related': True, 'reason': '留言提到的「幫你帶跑200公里」功能，與影片中提到的DeFit APP提供自動生成步數功能相關。', 'match_keywords': ['DeFit', '幫你帶跑超過200公里']}
==> 處理第8979則留言：烂梗是真low 扭曲部分信息还严肃讲出来误导观众 不是蠢就是坏
{'related': True, 'reason': '留言提到『烂梗』和『误导观众』，與影片中批評政治人物形象和言論的部分相關，例如批評賴清德和韓國瑜，以及提到的政治人物行為。留言反映了對影片中政治諷刺內容的態度，與影片主題關鍵字『賴清德』、『韓國瑜』相關。', 'match_keywords': ['賴清德', '韓國瑜']}
==> 處理第8980則留言：贺珑不用道歉
{'related': True, 'reason': '留言中的「贺珑」是影片主持人賀瓏，與影片主題關鍵字中的賀瓏相關。', 'match_keywords': ['賀瓏']}
==> 處理第8981則留言：XD 光一個黃珊珊 就捐個2百萬 財團真的猛 且絕對不是單押 捐款支持合法 但如有對政策指手劃腳 就絕非人民樂見
{'related': True, 'reas

In [24]:
test_df.to_csv("test_v0_related.csv", index=False, encoding='utf-8-sig')